In [ ]:
%matplotlib inline

## Make png outputs for all derived products for team QA before data release

In [ ]:
from pathlib import Path

from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from astropy.io import fits

from spectral_cube import Projection, SpectralCube

In [ ]:
derived_path = Path("/reduction/erickoch/LGLBS/line_imaging/derived/")

plot_path = derived_path / "derived_plots"
plot_path.mkdir(exist_ok=True)

In [ ]:
# Loop through all targets.

for this_target_path in derived_path.iterdir():

    this_target = this_target_path.name

    if this_target == plot_path.name:
        continue

    # Skip list:
    skips = ['ic10', 'ic1613', 'm33', 'ngc6822', 'wlm']
    if this_target in skips:
        print(f"Skipping {this_target}")
        continue

    print(this_target)

    # Make a new plot directory.

    this_plot_path = plot_path / this_target
    this_plot_path.mkdir(exist_ok=True)


    # Loop through products that exist in this directory
    for this_file in tqdm(this_target_path.glob("*.fits")):

        hdu = fits.open(this_file)[0]

        # Keep the moment maps only for now.
        if len(hdu.shape) == 2:

            proj = Projection.from_hdu(hdu)

            plt.figure(layout='constrained')
            plt.imshow(proj.value, origin='lower')
            cbar = plt.colorbar()
            cbar.set_label(proj.unit)
            plt.title(this_file.name)
            plt.savefig(this_plot_path / this_file.name.replace(".fits", ".jpg"),
                        dpi=300,
                        bbox_inches='tight')
            plt.close()

        else:

            out_filename = this_plot_path / this_file.name.replace(".fits", ".gif")
            if out_filename.exists():
                print(f"Skipping {out_filename}")
                continue

            # continue
            cube = SpectralCube.read(this_file).with_spectral_unit(u.km / u.s, 'radio')
            cube.allow_huge_operations = True


            fig, ax = plt.subplots(layout='constrained')

            fig.suptitle(this_file.name)

            artists = []
            for ii, vel in enumerate(cube.spectral_axis):
                if ii == 0:
                    animated = False
                else:
                    animated = True

                im = ax.imshow(cube.unitless_filled_data[ii],
                               origin='lower',
                               animated=animated)
                an = ax.annotate(f"{vel.value:.1f} {vel.unit}",
                                 xy=(0.9, 0.9),
                                 xytext=(0.9, 0.9),
                                 xycoords='axes fraction',
                                 size=14,
                                 ha='right',
                                 va='bottom',
                                 bbox=dict(facecolor='white', edgecolor='k', boxstyle='round'))
                artists.append([im, an])


            ani = animation.ArtistAnimation(fig, artists,
                                            interval=200,
                                            blit=True,
                                            repeat_delay=1000)

            writergif = animation.PillowWriter(fps=5)
            ani.save(out_filename,
                     writer=writergif)

            plt.close()
